<h3>Sentiment analysis of 50k IMDB movie reviews</h3>

In [ ]:
import pandas as pd
import numpy as np
import nltk
 
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")
nltk.download('punkt_tab')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.tokenize.toktok import ToktokTokenizer


from bs4 import BeautifulSoup
import re

data = pd.read_csv('IMDB Dataset.csv')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ognjen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ognjen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ognjen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Ognjen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [49]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


### Text Preprocessing Steps - removing html syntax

In [50]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

#Apply function on review column
data['review']=data['review'].apply(denoise_text)

C:\Users\Ognjen\AppData\Local\Temp\ipykernel_14784\652816855.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [51]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

#Apply function on review column
data['review']=data['review'].apply(remove_special_characters)

In [75]:
#sentiment count
data['sentiment'].value_counts()


sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [68]:
data['tokenized_review'] = data['review'].str.lower().apply(word_tokenize)

In [70]:
tokens = []
for review in data['tokenized_review']:
    tokens.extend(review)


In [71]:
stop_words = set(stopwords.words('english'))
filtered_tokens = [word for word in tokens if word.lower() not in stop_words and word.isalpha()]

word_freq = FreqDist(filtered_tokens)

# 12 most common words excluding first two since they are 'movie' and 'film'
most_common_words = [word for word, freq in word_freq.most_common()[2:14]]

In [72]:
most_common_words

['one',
 'like',
 'good',
 'even',
 'would',
 'time',
 'really',
 'see',
 'story',
 'much',
 'well',
 'get']

In [74]:
# creating columns with most frequent words in the reviews
for word in most_common_words:
    data[word] = data['tokenized_review'].apply(lambda x: 1 if word in x else 0)

data.head(10)

,review,sentiment,tokenized_review,one,like,good,would,time,really,see,even,story,much,get,bad,well
0,One of the other reviewers has mentioned that ...,positive,"[one, of, the, other, reviewers, has, mentione...",1,0,0,1,0,0,0,0,0,0,1,0,1
1,A wonderful little production The filming tech...,positive,"[a, wonderful, little, production, the, filmin...",1,0,0,0,0,1,1,0,0,0,0,0,1
2,I thought this was a wonderful way to spend ti...,positive,"[i, thought, this, was, a, wonderful, way, to,...",1,0,0,0,1,0,1,1,0,0,0,0,1
3,Basically theres a family where a little boy J...,negative,"[basically, theres, a, family, where, a, littl...",0,1,0,0,0,0,1,0,0,0,0,0,1
4,Petter Matteis Love in the Time of Money is a ...,positive,"[petter, matteis, love, in, the, time, of, mon...",1,0,1,0,1,0,1,0,0,0,0,0,0
5,Probably my alltime favorite movie a story of ...,positive,"[probably, my, alltime, favorite, movie, a, st...",1,1,0,0,0,0,0,0,1,0,0,0,0
6,I sure would like to see a resurrection of a u...,positive,"[i, sure, would, like, to, see, a, resurrectio...",0,1,0,1,0,0,1,0,0,0,0,0,0
7,This show was an amazing fresh innovative ide...,negative,"[this, show, was, an, amazing, fresh, innovati...",1,0,0,0,1,1,1,0,0,0,0,1,0
8,Encouraged by the positive comments about this...,negative,"[encouraged, by, the, positive, comments, abou...",1,0,0,0,0,0,1,0,0,0,0,0,0
9,If you like original gut wrenching laughter yo...,positive,"[if, you, like, original, gut, wrenching, laug...",0,1,0,0,0,0,0,1,0,0,0,0,0
